In [10]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime

In [11]:
df = pd.read_csv("../data/raw/itineraires_alpes.csv")
print(f"Chargés : {len(df)} itinéraires")
df.head()

Chargés : 161 itinéraires


,name,massif,lat,lon,denivele_positif,exposition,difficulty_ski
0,Aiguille du Midi,Chamonix,45.9234,6.8683,1200,N,S3
1,Col des Flambeaux,Chamonix,45.9170,6.8900,800,E,S2
2,Vallée Blanche,Chamonix,45.9100,6.8800,600,S,S1
3,Argentière Glacier,Chamonix,45.9500,6.9500,1500,N,S4
4,Pointe Helbronner,Chamonix,45.8300,6.9000,1000,SE,S3


In [12]:
BULLETINS_AVY = {
    "Chamonix": 3,
    "Vanoise": 2,
    "Écrins": 3,
    "Suisse": 3,
    "Italie": 2
}

def get_avy_risk(massif):
    return BULLETINS_AVY.get(massif, 3) / 5.0

In [13]:
# 3. Fonction vent OpenWeather (exemple simplifié – on prend un point central par massif)
WIND_CACHE = {}  # Pour éviter 150 appels API
def get_wind_score(lat, lon):
    key = f"{lat},{lon}"
    if key in WIND_CACHE:
        return WIND_CACHE[key]
    try:
        url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={open('../.env').read().strip().split('=')[1]}&units=metric"
        data = requests.get(url, timeout=5).json()
        wind = data["wind"]["speed"]  # m/s
        gust = data.get("wind", {}).get("gust", wind)
        wind_score = min(gust / 20.0, 1.0)  # >20 m/s = max pénalité
        WIND_CACHE[key] = wind_score
        return wind_score
    except:
        return 0.5  # pénalité moyenne si erreur

# 4. Fonction scoring finale v2 (la vraie, celle qu’on veut)
def scoring_v2(row, niveau="S3", dplus_max=1400):
    # --- DANGER ---
    avy_risk = get_avy_risk(row["massif"])
    
    # Vent sécurisé
    try:
        wind_score = get_wind_score(row["lat"], row["lon"])
    except:
        wind_score = 0.5
    
    expo_penalty = {
        "N":0.1, "NE":0.2, "E":0.4, "SE":0.7, "S":1.0,
        "SO":0.8, "O":0.6, "NO":0.3, "NW":0.2
    }.get(row["exposition"].strip(), 0.5)
    
    slope_penalty = 1.0 if row["difficulty_ski"] in ["S4","S5"] else 0.3
    
    danger_score = (
        0.40 * avy_risk +
        0.25 * wind_score +
        0.20 * expo_penalty +
        0.15 * slope_penalty
    )
    
    # --- FITNESS ---
    diff_level = abs(["S1","S2","S3","S4","S5"].index(row["difficulty_ski"]) - 
                     ["S1","S2","S3","S4","S5"].index(niveau))
    fitness = (dplus_max / row["denivele_positif"]) * (1 / (1 + diff_level))
    
    final_score = fitness / (1 + danger_score)
    
    # LE RETURN QUI MANQUAIT !!!
    return final_score, danger_score, avy_risk*5, wind_score*20

print("Fonction scoring_v2 corrigée avec return")

Fonction scoring_v2 corrigée avec return


In [16]:
print("Calcul en cours...")
results = df.apply(scoring_v2, axis=1, args=("S3", 1400))
df["final_score"], df["danger"], df["avy_risk"], df["wind_kmh"] = zip(*results)
print("Calcul terminé !")

Calcul en cours...
Calcul terminé !


In [17]:
top10 = df.sort_values("final_score", ascending=False).head(10)
display(top10[["name", "massif", "denivele_positif", "exposition", 
               "difficulty_ski", "avy_risk", "wind_kmh", "danger", "final_score"]].round(2))

,name,massif,denivele_positif,exposition,difficulty_ski,avy_risk,wind_kmh,danger,final_score
139,Col du Grand Paradis,Italie,900,NE,S3,2.0,5.61,0.32,1.18
110,Col de Checroui,Suisse,900,NE,S3,3.0,0.51,0.33,1.17
101,Col de Valpelline,Suisse,900,NE,S3,3.0,6.69,0.41,1.10
119,Col de la Dixence,Suisse,1000,NE,S3,3.0,0.51,0.33,1.05
53,Dôme de la Sache,Vanoise,1100,N,S3,2.0,2.83,0.26,1.01
46,Dôme des Nants,Vanoise,1100,NE,S3,2.0,2.16,0.27,1.00
36,Pointe de la Réchasse,Vanoise,1100,NE,S3,2.0,3.03,0.28,0.99
99,Pigne d'Arolla,Suisse,1000,E,S3,3.0,3.73,0.41,0.99
64,Dôme des Platières,Vanoise,1100,E,S3,2.0,2.51,0.32,0.97
4,Pointe Helbronner,Chamonix,1000,SE,S3,3.0,3.30,0.47,0.95
